# 08_Making_and_using_hostart_files.ipynb

## Using PAVICS-Hydro to create a hotstart file to resume a simulation from given hydrological conditions

Here we run a hydrological model with `Raven` over a long period. Then we run the model on half of the period, save the final states, and then run the second half in an independent run but using the model states at the end of the first half. This should provide two (2) hydrographs that are exactly equal.

In [ ]:
import datetime as dt
import xarray as xr
from ravenpy.utilities.testdata import get_file

## A note on datasets

For this introduction to RavenPy, we will use pre-existing datasets that are hosted on the PAVICS-Hydro servers, as we did in the previous example notebook. However, this time the model will not be pre-configured: We will configure it on the fly! In the next tutorials, we will see how users can import and use their own datasets to make the entire process flexible and taylored the user needs.

In [ ]:
forcing = get_file("raven-gr4j-cemaneige/Salmon-River-Near-Prince-George_meteo_daily.nc")

# Display the datasets that we will be using
display(forcing)

## Using templated model emulators

`Raven` framework can be used to emulate existing hydrological models. The Python wrapper offers at present four emulated models: GR4J-CN, HMETS, MOHYSE and HBV-EC. For each of these, templated configuration files are available to facilitate launching the model with options passed by Python at run-time. In this block of code, we are going to run the GR4JCN model and provide a list of configuration parameters. Please see the documentation for more details on the mandatory vs optional parameters, and what they represent. A glimpse is provided here.

In [ ]:
# Import the list of possible model templates. 
from ravenpy.models import GR4JCN, HMETS, MOHYSE, HBVEC

# Generate a GR4JCN-configured Raven model instance. 
# By replacing "GR4JCN()" by "HMETS()", we would then be running a HMETS model emulator instead. 
model = GR4JCN()

# Here is where we launch the model using all sorts of configuration parameters. 
# We will run the code, and explanations will follow.
model(
    forcing,
    start_date=dt.datetime(2000, 1, 1),
    end_date=dt.datetime(2001, 12, 31),
    area=4250.6,
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    params=(0.529, -3.396, 407.29, 1.072, 16.9, 0.947),
    run_name="Salmon",
)

In [ ]:
first_hydrograph = model.hydrograph.q_sim 

### Now let's see if we can replicate the hydrograph using a hotstart file. 
First, run the model without a hotstart file, same as before, but only for one year:


In [ ]:
model = GR4JCN()

model(
    forcing,
    start_date=dt.datetime(2000, 1, 1),
    end_date=dt.datetime(2000, 12, 31),
    area=4250.6,
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    params=(0.529, -3.396, 407.29, 1.072, 16.9, 0.947),
    run_name="Salmon",
)

second_hydrograph_first_year = model.hydrograph.q_sim

# Get the hotstart file path
hotstart_states = model.outputs["solution"] 


Now, let's run the model using the hotstart but for the second year only:

In [ ]:
model = GR4JCN()

# Here is where we launch the model using all sorts of configuration parameters. We will run the code, and explanations will follow.
model(
    forcing,
    start_date=dt.datetime(2000, 12, 31),
    end_date=dt.datetime(2001, 12, 31),
    area=4250.6,
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    params=(0.529, -3.396, 407.29, 1.072, 16.9, 0.947),
    run_name="Salmon",
    rvc=str(hotstart_states)
)

second_hydrograph_second_year = model.hydrograph.q_sim

In [ ]:
second_hydrograph_complete = xr.concat([
    second_hydrograph_first_year[0:-1],
    second_hydrograph_second_year],
    'time'
)

## Plotting both hydrographs
Let's plot both hydrographs to see if they are the same:

In [ ]:
first_hydrograph.plot()
second_hydrograph_complete.plot()

And now if we look at the difference between both hydrographs:

In [ ]:
difference = first_hydrograph - second_hydrograph_complete
difference.plot()

We can see that differences are at machine precision levels, due to rounding in the hotstart file (note that the y-axis is 1e-6, which is essentially 0!). But the rest is perfect!